# Installs

### Instructions to run the code

Running this notebook would ensure that the model runs on the data provided, The data for this homework comes from 11785 course at Carnegie Mellon university. The specifics are written before each and every main block:

The code follows the following structure:
* Importing the required packages
* Defining the Dataloaders to use it for training and testing
* Checking the dataloaders
* Defining the model
* Training and evaluation loop
* Training
* Generating submissions


## wandb

You will need to fetch your api key from wandb.ai

In [ ]:
!nvidia-smi

Tue Nov 22 04:00:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb -q

     |████████████████████████████████| 1.9 MB 5.0 MB/s 
     |████████████████████████████████| 168 kB 83.1 MB/s 
     |████████████████████████████████| 182 kB 84.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 86.3 MB/s 
     |████████████████████████████████| 166 kB 92.4 MB/s 
     |████████████████████████████████| 162 kB 95.0 MB/s 
     |████████████████████████████████| 162 kB 83.0 MB/s 
     |████████████████████████████████| 158 kB 85.7 MB/s 
     |████████████████████████████████| 157 kB 95.3 MB/s 
     |████████████████████████████████| 157 kB 88.8 MB/s 
     |████████████████████████████████| 157 kB 27.3 MB/s 
     |████████████████████████████████| 157 kB 89.7 MB/s 
     |████████████████████████████████| 157 kB 93.7 MB/s 
     |████████████████████████████████| 157 kB 97.0 MB/s 
     |████████████████████████████████| 157 kB 88.6 MB/s 
     |████████████████████████████████| 156 kB 94.0 MB/s 


In [ ]:
import wandb
wandb.login(key="e88e5782628fb1039c9df3e4f75ae46a961a0979")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Levenshtein

This may take a while

In [ ]:
!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

!pip install torchsummaryX

     |████████████████████████████████| 175 kB 5.0 MB/s 
     |████████████████████████████████| 2.2 MB 74.6 MB/s 
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 12.42 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14102, done.        
remote: Counting objects: 100

## imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"kwaghela04","key":"02aa393f704d522832a2ea55ec85f87a"}')

!chmod 600 /root/.kaggle/kaggle.json

  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c 11-785-f22-hw3p2

100% 8.87G/8.88G [00:51<00:00, 257MB/s]
100% 8.88G/8.88G [00:51<00:00, 184MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''
!unzip -q 11-785-f22-hw3p2.zip
!ls

11-785-f22-hw3p2.zip  ctcdecode  hw3p2	sample_data


# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ", # BLANK TOKEN
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict
mapping = CMUdict_ARPAbet
LABELS = ARPAbet

In [ ]:
# You might want to play around with the mapping as a sanity check here

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, partition='train-clean-360', context=0, offset=0): 
        self.mfcc_dir = 'hw3p2/'+partition+'/mfcc'
        self.transcript_dir = 'hw3p2/'+partition+'/transcript/raw'

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES

        self.mfccs, self.transcripts = [], []

        for i in range(0, len(self.mfcc_files)):
            mfcc = np.load(self.mfcc_dir+"/"+self.mfcc_files[i])
            mfcc = (mfcc - np.mean(mfcc, axis=0))/np.std(mfcc, axis=0)
            transcript = np.delete(np.load(self.transcript_dir+"/"+self.transcript_files[i]), (0, -1)) 
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        self.length = len(self.mfccs)
        t = []
        for tr in self.transcripts:
          t.append(np.array([self.PHONEMES.index(i) for i in tr]))
        self.transcripts = t

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[ind] 
        frames = torch.FloatTensor(frames) # Convert to tensors
        phoneme = torch.tensor(self.transcripts[ind])       

        return frames, phoneme


    def collate_fn(self,batch):
        batch_mfcc = [b[0] for b in batch] # TODO
        batch_transcript = [b[1] for b in batch] # TODO
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True, padding_value=0.0) # TODO
        lengths_mfcc = [len(m) for m in batch_mfcc] # TODO 
        batch_transcript_pad = torch.nn.utils.rnn.pad_sequence(batch_transcript, batch_first=True, padding_value=0.0) # TODO
        lengths_transcript = [len(t) for t in batch_transcript] # TODO
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

### Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
   def __init__(self, partition='test-clean'): 

        self.mfcc_dir = 'hw3p2/'+partition+'/mfcc'
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.PHONEMES = PHONEMES
        self.length = len(self.mfcc_files)
        self.mfccs, self.transcripts = [], []
        for i in range(0, len(self.mfcc_files)):
            mfcc = np.load(self.mfcc_dir+"/"+self.mfcc_files[i])
            mfcc = (mfcc - np.mean(mfcc, axis=0))/np.std(mfcc, axis=0)
            self.mfccs.append(mfcc)

        self.length = len(self.mfccs)

   def __len__(self):
    return self.length

   def __getitem__(self, ind):
        
        frames = self.mfccs[ind] 
        frames = torch.FloatTensor(frames) # Convert to tensors     
        return frames

   def collate_fn(self,batch):
        batch_mfcc = [b for b in batch] # TODO
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True, padding_value=0.0) # TODO
        lengths_mfcc = [len(m) for m in batch_mfcc] # TODO 
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

### Data - Hyperparameters

In [ ]:
BATCH_SIZE = 64 # Increase if your device can handle it

transforms = [] # set of tranformations

### Data loaders

In [ ]:

import gc 
gc.collect()

264

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset() #TODO
val_data = AudioDataset(partition='dev-clean') 
test_data = AudioDatasetTest() #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=BATCH_SIZE, pin_memory= True,
                                           shuffle= True, collate_fn=train_data.collate_fn)
val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=BATCH_SIZE, pin_memory= True,
                                         shuffle= False, collate_fn=val_data.collate_fn)
test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=BATCH_SIZE, pin_memory= True, 
                                          shuffle= False, collate_fn=test_data.collate_fn)

print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 104014, batches = 1626
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([64, 1651, 15]) torch.Size([64, 186]) torch.Size([64]) torch.Size([64])


In [ ]:
for data in val_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([64, 1907, 15]) torch.Size([64, 217]) torch.Size([64]) torch.Size([64])


In [ ]:
for data in test_loader:
    x, lx = data
    print(x.shape, lx.shape)
    break 

torch.Size([64, 825, 15]) torch.Size([64])


# Model Config

In [ ]:
OUT_SIZE = len(LABELS)
OUT_SIZE

43

## Defining the Model

This class is defined for locked dropout regularization

In [ ]:
from torch.autograd import Variable
import sys
class LockedDropout(nn.Module):
    def forward(self, x, dropout=0.2):
        x, lx = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
        m = x.data.new(x.size(0), 1, x.size(2)).bernoulli_(1 - dropout)
        mask = Variable(m, requires_grad=False) / (1 - dropout)
        mask = mask.expand_as(x)
        out = mask * x
        return torch.nn.utils.rnn.pack_padded_sequence(out, lengths=lx, batch_first=True, enforce_sorted=False)

A final model with a CNN embedding layer followed by a bidirectional block and finally a linear classification unit for final classifications

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()
        self.embedding = nn.Sequential(
              nn.Conv1d(15, 256, 1),
              nn.BatchNorm1d(256),
              nn.GELU(),

              nn.Conv1d(256, 512, 1),
              nn.BatchNorm1d(512),
              nn.GELU(),

              nn.Conv1d(512, 512, 1),
              nn.BatchNorm1d(512),
              nn.GELU(),
        )
        self.lstms = nn.Sequential(
            nn.LSTM(input_size = 512, hidden_size = 256, num_layers = 4, bidirectional=True, dropout=0.2, batch_first=True),
        )
        self.classification = nn.Sequential(
            nn.Linear(512, 256),nn.Dropout(0.2),
            nn.GELU(),
            nn.Linear(256, 256),
            nn.GELU(),
            nn.Linear(256, OUT_SIZE)
        )
        self.logSoftmax = nn.LogSoftmax(dim=2)

    def forward(self, x, lx):
        x = torch.permute(x, (0,2,1))
        x = torch.permute(self.embedding(x), (0,2,1))
        packed = torch.nn.utils.rnn.pack_padded_sequence(x, lengths=lx, batch_first=True, enforce_sorted=False)
        out,_ = self.lstms(packed)
        unpacked,len_unpacked = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        out = self.classification(unpacked)
        out = self.logSoftmax(out)
        return out, len_unpacked

## INIT

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                              Kernel Shape    Output Shape    Params  \
Layer                                                                  
0_embedding.Conv1d_0          [15, 256, 1]  [64, 256, 825]    4.096k   
1_embedding.BatchNorm1d_1            [256]  [64, 256, 825]     512.0   
2_embedding.GELU_2                       -  [64, 256, 825]         -   
3_embedding.Conv1d_3         [256, 512, 1]  [64, 512, 825]  131.584k   
4_embedding.BatchNorm1d_4            [512]  [64, 512, 825]    1.024k   
5_embedding.GELU_5                       -  [64, 512, 825]         -   
6_embedding.Conv1d_6         [512, 512, 1]  [64, 512, 825]  262.656k   
7_embedding.BatchNorm1d_7            [512]  [64, 512, 825]    1.024k   
8_embedding.GELU_8                       -  [64, 512, 825]         -   
9_lstms.LSTM_0                           -    [28434, 512]  6.30784M   
10_classification.Linear_0      [512, 256]  [64, 825, 256]  131.328k   
11_classification.Dropout_1              -  [64, 825, 256]      

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding.Conv1d_0,"[15, 256, 1]","[64, 256, 825]",4096.0,3168000.0
1_embedding.BatchNorm1d_1,[256],"[64, 256, 825]",512.0,256.0
2_embedding.GELU_2,-,"[64, 256, 825]",NaN,NaN
3_embedding.Conv1d_3,"[256, 512, 1]","[64, 512, 825]",131584.0,108134400.0
4_embedding.BatchNorm1d_4,[512],"[64, 512, 825]",1024.0,512.0
5_embedding.GELU_5,-,"[64, 512, 825]",NaN,NaN
6_embedding.Conv1d_6,"[512, 512, 1]","[64, 512, 825]",262656.0,216268800.0
7_embedding.BatchNorm1d_7,[512],"[64, 512, 825]",1024.0,512.0
8_embedding.GELU_8,-,"[64, 512, 825]",NaN,NaN


# Training Config

In [ ]:
train_config = {
    "beam_width" : 4,
    "lr" : 2e-3,
    "epochs" : 80
    } # Feel free to add more items here

In [ ]:
criterion = nn.CTCLoss()
lr = 1e-3
optimizer = torch.optim.AdamW(params = model.parameters(), lr = lr, weight_decay = 2e-3)
decoder = CTCBeamDecoder(labels = PHONEMES, beam_width = 4, log_probs_input = True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer = optimizer, mode='min', factor = 0.5, patience = 4, threshold = 0.01)

### Levenshtein

In [ ]:
def calculate_levenshtein(h, y, lh, ly, decoder, PHONEME_MAP):
    batch_size = h.shape[0]
    dist = 0
    beam_results, beam_scores, timesteps, out_lens = decoder.decode(h, seq_lens = lh)
    for i in range(batch_size): 
        beam = beam_results[i][0][:out_lens[i][0]]

        h_string = "".join([PHONEME_MAP[x] for x in beam])

        y_sliced = y[i,0:ly[i]]
        y_string = "".join([PHONEME_MAP[x] for x in y_sliced])
        
        dist += Levenshtein.distance(h_string, y_string)

    dist /= batch_size

    return dist

In [ ]:
run = wandb.init(
    name = "hw3-med", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = train_config ### Wandb Config for your run
)

# Training

### Eval function
Writing a function to do one round of evaluations will help make your code more modular, you can, however, choose to skip this if you'd like it.

In [ ]:
scaler = torch.cuda.amp.GradScaler()

def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    for i, data in (enumerate(train_loader)):
        
        optimizer.zero_grad()

        x, y, lx, ly = data
        x = x.to(device)
        y = y.to(device)

        with torch.cuda.amp.autocast():
            outputs, opLength = model(x, lx)
            outputs = torch.permute(outputs, (1,0,2))
            loss = criterion(outputs, y, opLength, ly)
        
        total_loss += loss.item()
        bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))))
        scaler.scale(loss).backward() 
        scaler.step(optimizer) 
        scaler.update()
        bar.update()

    total_loss /= len(train_loader)

    return total_loss

def validate(model, val_loader, optimizer, criterion):
    model.eval()
    total_loss = 0
    distance = 0
    bar = tqdm(total=len(val_loader), dynamic_ncols=True, leave=False, position=0, desc='Validation')
    for i, data in (enumerate(val_loader)):
        
        optimizer.zero_grad()

        x, y, lx, ly = data
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():
            outputs, opLength = model(x, lx)
            outputs = torch.permute(outputs, (1,0,2))
            loss = criterion(outputs, y, opLength, ly)
        
        total_loss += loss.item()
        outputs = torch.permute(outputs, (1,0,2))
        distance += calculate_levenshtein(outputs, y, opLength, ly, decoder, LABELS) 
        bar.update()

    total_loss /= len(val_loader)
    distance /= len(val_loader)
    return total_loss, distance

In [ ]:
for i in range(1,61):
    print('Epoch', i)
    torch.cuda.empty_cache()

    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_distance = validate(model, val_loader, optimizer, criterion)
    
    scheduler.step(val_distance)
    print(scheduler._last_lr)

    if i > 20 and i % 5 == 0:
        checkpoint = { 
            'epoch': i,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_sched': scheduler
        }
        prev_loss = val_loss
        # torch.save(checkpoint, '/content/drive/checkpoint.pth')

    print('Training Loss', train_loss)
    print('Validation Loss', val_loss)
    print('Validation distance', val_distance)

Epoch 1


[0.0005]
Training Loss 0.16521841905287124
Validation Loss 0.2596424928931303
Validation distance 4.804408914728683
Epoch 2


[0.0005]
Training Loss 0.16081389641849758
Validation Loss 0.2573622667858767
Validation distance 4.744791666666666
Epoch 3


[0.0005]
Training Loss 0.1570687897292597
Validation Loss 0.24851747342320377
Validation distance 4.580813953488372
Epoch 4


[0.0005]
Training Loss 0.15406143504021938
Validation Loss 0.2527657731674438
Validation distance 4.690625
Epoch 5


...

[0.0005]
Training Loss 0.1365125044438012
Validation Loss 0.2533000716289809
Validation distance 4.574636627906977
Epoch 12


[0.0005]
Training Loss 0.13603902866131412
Validation Loss 0.25401984224485796
Validation distance 4.503076550387597
Epoch 13


[0.00025]
Training Loss 0.13280361712088884
Validation Loss 0.2539481134262196
Validation distance 4.4984738372093025
Epoch 14


Train:  23%|██▎       | 380/1626 [07:01<22:51,  1.10s/it, loss=0.1219]

KeyboardInterrupt: ignored

# Generate Predictions and Submit to Kaggle

In [ ]:
decoder_test = CTCBeamDecoder(LABELS, beam_width=train_config['beam_width'], log_probs_input=True)

def make_output(h, lh, decoder, LABELS):
  
    beam_results, beam_scores, timesteps, out_seq_len = decoder.decode(h, lh) #TODO: What parameters would the decode function take in?
    batch_size = h.shape[0]

    dist = 0
    preds = []
    for i in range(batch_size): # Loop through each element in the batch

        h_sliced = beam_results[i,0,:out_seq_len[i,0]] #TODO: Obtain the beam results
        h_string = ''.join(LABELS[j] for j in h_sliced) #TODO: Convert the beam results to phonemes
        preds.append(h_string)
    
    return preds

In [ ]:
def predict(test_loader, model, decoder, LABELS):
    model.eval()
    batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, position=0, leave=False, desc='Test')
    predictions = []

    for i, data in enumerate(tqdm(test_loader)):
        m_pad, m_len = data
        m_pad = m_pad.to(device)

        with torch.inference_mode():
            out, out_lengths = model(m_pad, m_len)
            preds = make_output(out, out_lengths, decoder, LABELS)

        predictions.extend(preds)

        batch_bar.update()

        del m_pad, m_len
        del preds
        torch.cuda.empty_cache()
  
    batch_bar.close()
    return predictions

In [ ]:
torch.cuda.empty_cache()
predictions = predict(test_loader, model, decoder, LABELS)
import pandas as pd

df = pd.read_csv('/content/hw3p2/test-clean/transcript/random_submission.csv')
df.label = predictions

df.to_csv('submission.csv', index = False)
# !kaggle competitions submit -c 11-785-f22-hw3p2 -f submission.csv -m "Message" "I made it!"

100%|██████████| 41/41 [00:35<00:00,  1.15it/s]


403 - Forbidden


In [ ]:
!kaggle competitions submit -c 11-785-f22-hw3p2-slack -f submission.csv -m "Message"

100% 209k/209k [00:00<00:00, 1.01MB/s]
Successfully submitted to Automatic Speech Recognition (ASR - SLACK)